# Covid Sales Forecasting

### 목차

#### 1. 모듈 및 데이터 임포트
#####  <GPU 확인>
#####  <데이터 임포트>
<br/>

#### 2. 데이터 전처리
#####  2.1. MinMaxScaling
#####  2.2. 데이터 연관성 분석
#####  2.3. 이상치 처리
#####  2.4. 데이터셋 분할
#####  <결과기록용 데이터셋 저장>
#####  <테스트셋 csv 저장>
<br/>

#### 3. 딥러닝
#####  3.1. 네트워크 모델
#####  <Plot model 저장>
#####  3.2. 학습
<br/>

#### 4. 결과 표시
#####  4.1. 손실 함수 그래프
#####  <채점용 csv 저장>
#####  4.2. 측정 성능 출력
#####  4.3. 예측 그래프

##### <화면 가로 확장>

In [1]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### 1. 모듈 임포트

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import tensorflow as tf
import csv
import os
import datetime
import h5py

##### <GPU 확인>

In [ ]:
tf.test.gpu_device_name()

#### 최종 데이터 Import

In [ ]:
Data_Combined=pd.read_csv('./Data/2021_Data_Combined.csv', encoding='cp949')
Data_Combined

In [ ]:
Data_Combined['Won'].max()

### 2. 데이터 전처리

#### 2.1. Min-Max Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# 정규화 과정을 거쳐 데이터 범위를 원하는 범위로 제한. 그래프는 동일하나 손실 정도를 확인하기에 더 좋음.
# 배치 정규화는 기울기 소멸 문제를 해결하기는 좋으나, RNN의 경우 계층별로 미니 정규화 적용이 필요해 모델이 복잡해지고 효율이 저하됨.
Data_Combined.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['확진자수(양성)', '검사수', '확진자 증가량', '검사수 증가량', '18시 전 규제 인원수', '18시 후 규제 인원수', '영업 제한 시각', '평균기온(°C)', '강수여부', '일강수량(mm)', '평균 풍속(m/s)', 'Won']
Data_scaled = scaler.fit_transform(Data_Combined[scale_cols])
Data_scaled = pd.DataFrame(Data_scaled)
Data_scaled.columns = scale_cols

Data_scaled.insert(0, '날짜', Data_Combined['날짜'])
Data_scaled

#### 2.2.. 데이터 연관성 분석

In [ ]:
Data_scaled.columns

In [ ]:
import scipy.stats as stats

# 피어슨 stat을 이용하면 피어슨 상관계수와 p-value값이 출력되는데 상관계수는 양 or 음의 비례관계 정도를, p-value는 상관 정도를 보여준다. (0에 가까울수록 영향이 크다.)
def P_value(dataset, c1, c2):
    Correlation=stats.pearsonr(dataset[c1], dataset[c2])
    print('< '+c1+'-'+c2+' >')
    print('P-value ==>> %.3f\n' %Correlation[1])

for p in range(1, len(Data_scaled.columns)-1):
    P_value(Data_scaled, Data_scaled.columns[p], 'Won')

In [ ]:
#Data_P = Data_scaled.drop([Data_scaled.columns[1], Data_scaled.columns[3], Data_scaled.columns[4], Data_scaled.columns[5], Data_scaled.columns[7], Data_scaled.columns[9]], axis=1)
Data_P = Data_scaled.loc[:, ['날짜', '검사수', '검사수 증가량', '영업 제한 시각', '평균기온(°C)', '일강수량(mm)', 'Won']]

In [ ]:
Data_P

#### 2.3. 이상치 처리

In [ ]:
def get_outlier(df=None, column=None, weight=None):
    quantile_5 = np.percentile(df[column].values, 5)
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_50 = np.percentile(df[column].values, 50)
    quantile_75 = np.percentile(df[column].values, 75)
    quantile_95 = np.percentile(df[column].values, 95)
    
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR * weight

    lowest_val = quantile_25-IQR_weight
    highest_val = quantile_75+IQR_weight
    
    outlier_index_lowest = df[column][df[column] < quantile_5].index
    outlier_index_low = df[column][df[column] < lowest_val].index
    outlier_index_high = df[column][df[column] > highest_val].index
    outlier_index_highest = df[column][df[column] > quantile_95].index
    
    return outlier_index_lowest, outlier_index_low, outlier_index_high, outlier_index_highest, quantile_5, quantile_25, quantile_50, quantile_75, quantile_95

In [ ]:
WEIGHT=2.5
Lowest_list = get_outlier(df=Data_P, column='Won', weight=WEIGHT)[0]
Highest_list = get_outlier(df=Data_P, column='Won', weight=WEIGHT)[3]

print('Lowest Outlier ==>> {}\n'.format(Lowest_list))
print('Low Outlier ==>> {}\n'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[1]))
print('High Outlier ==>> {}\n'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[2]))
print('Highest Outlier ==>> {}\n'.format(Highest_list))

print('5% Value ==>> {}\n'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[4]))
print('25% Value ==>> {}\n'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[5]))
print('50% Value ==>> {}\n'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[6]))
print('75% Value ==>> {}\n'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[7]))
print('95% Value ==>> {}'.format(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[8]))

In [ ]:
WEIGHT_2R=np.sqrt(WEIGHT)

for i in range(len(Lowest_list)):
    Data_P['Won'][Lowest_list[i]] = Data_P['Won'][Lowest_list[i]]*WEIGHT_2R
    
for i in range(len(Highest_list)):
    Data_P['Won'][Highest_list[i]] = Data_P['Won'][Highest_list[i]]/WEIGHT_2R

In [ ]:
high_val = get_outlier(df=Data_P, column='Won', weight=WEIGHT)[2].values.tolist()

for i in range(len(high_val)):
    Data_P['Won'][high_val[i]] = Data_P['Won'][high_val[i]]/WEIGHT_2R

In [ ]:
print('Quantity of Low Outlier ==>> {}\n'.format(len(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[1])))
print('Quantity of High Outlier ==>> {}\n'.format(len(get_outlier(df=Data_P, column='Won', weight=WEIGHT)[2])))

#### 2.4. 데이터셋 분할

In [ ]:
TEST_SIZE = 14
WINDOW_SIZE = 7 # 얼마 동안의 과거 기반의 데이터에 기반하여 다음날의 값을 예측할 것인지 설정. 
BATCH_SIZE = 256

train = Data_P[:-TEST_SIZE]
test= Data_P[-TEST_SIZE:]
test = test.reset_index(drop=True) # 인덱스 초기화

In [ ]:
from sklearn.model_selection import train_test_split

feature_cols = ['검사수', '검사수 증가량', '영업 제한 시각', '평균기온(°C)', '일강수량(mm)', 'Won']
label_cols = ['Won']

train_feature = train[feature_cols]
train_label = train[label_cols]

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.3, shuffle=False)

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, False) # 학습도 셔플 안해야 맞는게 아닌가??
valid_data = windowed_dataset(y_valid, WINDOW_SIZE, BATCH_SIZE, False)

In [ ]:
for data in train_data.take(1):
    print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
    print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')

##### <결과기록용 데이터셋 저장>

In [ ]:
train
train.to_csv('./Result/score/Learning_set.csv', index=False)

##### <평가용 csv 저장>

In [ ]:
TESTSET = test['Won']
TESTSET.to_csv('./Result/score/Test_set.csv', index=False)

### 3. 딥러닝

#### 3.1. 네트워크 모델

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
import keras
LeakyReLU=tf.keras.layers.LeakyReLU(alpha=0.01)

#16-0.2-8-1
model = Sequential([
    Dense(8, activation='ELU', input_shape=[WINDOW_SIZE, 1]),
    Dense(8, activation='ELU'),
    Dense(2, activation='ELU'),
    Dense(1)
                    ])

model.summary()

model.compile(loss='mean_absolute_error', optimizer="Nadam", metrics=['mae'])

##### <Plot model 저장>

In [ ]:

from keras.utils.vis_utils import plot_model
import pydot
import pydotplus
import graphviz

plot_model(model, to_file='./Result/model.png')
plot_model(model, to_file='./Result/model_shapes.png', show_shapes=True)


#### 3.2. 학습

In [ ]:
#early_stop = EarlyStopping(monitor='val_loss', patience=20)

with tf.device('/GPU:0'):
    model_path = 'model'
    filename = os.path.join(model_path, 'tmp_checkpoint_Covid.h5')
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    history = model.fit(train_data, epochs=35, batch_size=BATCH_SIZE, validation_data=(valid_data), callbacks=[checkpoint])

In [ ]:
model.load_weights(filename)
pred = model.predict(test['Won'])

### 4. 결과 표시

In [ ]:
try:
    os.mkdir('./Result')
    print('폴더 생성 완료.')
    
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.')

In [ ]:
try:
    os.mkdir('./Result/Graph')
    os.mkdir('./Result/Data')
    print('폴더 생성 완료.')
    
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.')

#### 4.1. 손실 함수 그래프

In [ ]:
plt.figure(figsize=(16, 10))
plt.title('Model Loss Convergence Graph', size='15')
y_tloss = history.history['loss']
y_vloss = history.history['val_loss']
x_len = np.arange(len(y_tloss))
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.plot(x_len, y_tloss, "o-", c="blue", markersize=3)
plt.plot(x_len, y_vloss, "o-", c="red", markersize=3)
plt.margins(x=0.02)
plt.legend(['Training loss', 'Validation loss'])
plt.savefig('./Result/Graph/Model_Loss_Convergence_Graph.png')
plt.show

In [ ]:
pred=pred.reshape(-1,1)

In [ ]:
pred.shape

##### <채점용 csv 저장>

In [ ]:
PRED_df = pd.DataFrame(pred)
PRED_df.to_csv('./Result/score/Pred.csv', index=False)

#### 4.2. 측정 성능 출력

In [ ]:
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_squared_log_error as MSLE

RMSE = np.sqrt(MSE(test['Won'], pred))
R2 = r2(test['Won'], pred)
print('r2 >> %.4f' %R2) # 1에 가까워야 좋음
print('MAE >> %.4f' %MAE(test['Won'], pred)) # 0에 가까워야 좋음
print('RMSE >> %.4f' %RMSE) # 0에 가까워야 좋음

In [ ]:
RMSE_pct = round(1-RMSE, 5)*100
RMSE_Percentage='RMSE Accuracy = %.3f %%' %RMSE_pct
print(RMSE_Percentage)

In [ ]:
score = round(((1-RMSE)/2 + R2/2),4)
Final_score = 'Final Score = 0.5*r2 + 0.5*RMSE = %.4f' %score
print(Final_score)

#### 4.3. 예측 그래프

In [ ]:
plt.figure(figsize=(14,9))
plt.title('Sales Forecasting during COVID-19 ', y=1.03, size='15')
plt.suptitle(Final_score, y=0.9, fontsize=12)
plt.plot(test['Won'], label='Actual')
plt.plot(pred, label='Prediction')
plt.xticks(range(0,14), labels=range(1,15))
plt.xlabel('Days')
plt.yticks([0, 0.0688, 0.1376, 0.2064, 0.2752, 0.344, 0.4128], 
           labels=[0, 50000, 100000, 150000, 200000, 250000, 300000])
plt.ylabel('Won')
plt.grid(True)
plt.legend()
plt.savefig('./Result/Graph/Result_Grpah.png')
plt.show()